# Julia Extra! Self-induced stochastic resonance.

Continuing in this Julia series, let's say we're interested in examples where stochastic systems exhibit qualitatively different behavior than their deterministic/mean-field descriptions.

Consider the simple nonlinear reaction system given by:
$$
2A + B \rightarrow 3A $$
$$ \emptyset \rightleftharpoons A $$
$$ \emptyset \rightarrow B$$

Note that these are essentially just a library of rules for interactions, we have one set of "reversible" rules (eq 2), and two irreversible steps. The first step is the one considered "nonlinear" due to the fact that it depends on two terms one describing A and one describing B, and these aren't linearly additive as we'll see (from propensity functions).

In [ ]:
# make sure to have these packages installed
# you can do this by going to the Julia REPL, typing: "using Pkg", and then typing: "Pkg.add("PACKAGE NAME")"
# do this for the packages used in each of the notebooks.

using Distributions, Plots, LaTeXStrings, BenchmarkTools, ColorSchemes


We can model this system using the Gillespie Stochastic Simulation Algorithm (SSA). See the references for more information. In short, we can write "propensity functions" which are kind of like probability densities along intervals of time: the probability of a reaction $j$ occuring in the interval $dt$ is $f_{j} dt$. Now, let's write the probability distributions for this set of reactions:

$$ f_{1} = k_{1} A B (A - 1) $$
$$ f_{2} = k_{2} $$
$$ f_{3} = k_{3} A $$
$$ f_{4} = k_{4} $$

where we let $A = N(A)$, that is $A$ represents the number of type-$A$ molecules, and the same for B molecules. The rate of production of A, B doesn't depend on the existing population (per the rules, they just spawn at some rate).

To represent the state of the system, we can use a struct: a composite type in Julia. See the implementation below:

In [ ]:
mutable struct State
    """
    Structs are a composite type in Julia. We're using a struct to represent the state of the system.
    This is somewhat analgous to a class, but there are key differences since Julia is not an object-oriented language.

    The state of the system is represented by two integers, A and B, representing the molecule number of each species
    respectively.
    
    Note, unlike Python, we do have to care about the "mutability." I had to add the `mutable` keyword to the struct
    to be able to dynamically change the values of A and B, per the simulation. I have to explicitly declare that this 
    composite type is mutable, for changes to occur without error.
    """

    # Declare the type, and the names of the fields.
    A::Int64
    B::Int64

    # Overload the addition operator so that I can add things together willy nilly and not worry about
    # the types of the things I'm adding.

    Base.:+(x::State, y::State) = State(x.A + y.A, x.B + y.B)
    Base.:+(x::State, y::Tuple{Int64, Int64}) = State(x.A + y[1], x.B + y[2])
    Base.:+(x::Tuple{Int64, Int64}, y::State) = State(x[1] + y.A, x[2] + y.B)

    # Declare a constructor. I have declared multiple constructors here, so that I can initialize the state
    # in multiple ways. This is not strictly necessary, but it's nice to have.
    State(A0 :: Int64, B0 :: Int64) = new(A0, B0)
    State() = new(0, 0)
    State(A0 :: Int64) = new(A0, 0)
    State(B0 :: Int64) = new(0, B0)
    State(x :: Tuple{Int64, Int64}) = new(x[1], x[2])
end

We can write these propensities into Julia using lambda functions:

In [ ]:
rates = [4.0e-5, 50, 10, 25] # k1, k2, k3, k4 <- these are determined from quantum mechanics/theory of collisions.
                             # if you're modelling actual physical reactions, sec^-1 time-scale

# remember that indexing in Julis starts with 1
propensities = [
    (S::State) -> rates[1] * S.A * S.B * (S.A - 1), # <- this is what a lambda function looks like in Julia.
    (S::State) -> rates[2],                         # also note how I access the fields of the struct.
    (S::State) -> rates[3] * S.A,
    (S::State) -> rates[4]
]

# adding a semi-colon at the end of a line suppresses the output of that line.
change = [(1, -1), (1, 0), (-1, 0), (0, 1)]; # tuples that bring about the net change of a rxn (in the ordering of the rxns above)

#### Gillespie Stochastic Simulation.

We're going to use a method of stochastic simulation to generate "exact" (as opposed to mean) trajectories of this system. I've included a couple of good references of Gillespie simulation at the end, but the crux is that we are going to use a random draw to pick the next time, and next reaction to do.

To avoid increasing the length of this particular notebook, I'll go into detail about the Gillespie method in another notebook (and other cool vairants -- there are some cool algebra/data structures connections in the history of the Gillespie method). These variants include: Tau Leaping, Next-Step methods/using tree representations, etc.

Note the annotations (``::State, ::Float64`, and so on). These annotations restrict the use of step to this particular instance of a two component (A, B) reaction system. If I don't have that, my state definition breaks down and so does my stoichiometric change vector-- I would need to make both structures more general (include a dimension parameter) to create a step function capable of handling arbitrary components.

In [ ]:
function step(S::State, t::Float64, change::Vector{Tuple{Int64, Int64}}, propensities::Vector{Function})
    """
    This function takes in the current state of the system, and returns the next state of the system.
    This is the core of the Gillespie Stochastic Simulation Algorithm (SSA).
    Parameters:
    -----------
    S: State
        The current state of the system.
    t: Float64
        The current time of the system.
    change: Vector{Tuple{Int64, Int64}}
        The net change of each reaction.
    propensities: Vector{Function}
        The propensity functions of each reaction.
    
    Returns:
    ----------
    S: State
        The updated state of the system.
    t: Float64
        The updated time of the system.
    """
    a = [propensity(S) for propensity in propensities]
    a0 = sum(a)

    r1, r2 = rand(Uniform(0, 1), 2) # <- this is how you sample from a uniform distribution in Julia.
                                    # this line uses the Distributions.jl package to instantiate a uniform distribution
                                    # I am also implictily unpacking the tuple into two r1, r2 values.
    
    tol = r2 * a0
    a_sum = cumsum(a)
    
    tau = -log(r1) / a0             # Determine the next time
    j = findfirst(a_sum .> tol)     # Determine the next reaction index

    S += change[j]                  # Update the state of the system: this relies on the overloaded addition operator
    return S, t + tau
end;

In [ ]:
function simulate(S::State, tmax::Float64, change::Vector{Tuple{Int64, Int64}}, propensities::Vector{Function})
    """
    Simulates the system until tmax.
    Parameters:
    -----------
    S: State
        The initial state of the system.
    tmax: Float64
        The maximum time to simulate the system.
    change: Vector{Tuple{Int64, Int64}}
        The net change of each reaction.
    propensities: Vector{Function}
        The propensity functions of each reaction.
    
    Returns:
    ----------
    data: Vector{Tuple{Float64, State}}
        The time and state of the system at each time step.
    """
    t = 0.0
    data = [(t, S)]
    while t < tmax
        S, t = step(S, t, change, propensities)
        push!(data, (t, S))                         # in julia, the ! operator is used to indicate that the function
                                                    # modifies the input argument, so push! modifies/writes the data vector
    end
    return data
end;

Now, we have set up our simulator. We can simulate the time-evolution of this reacting system. Everything we have done has been very general, and you could imagine doing this for other "rule" schemes. The "chemical" reactions are even quite abstracted, and can be viewed generally as some potential paths that are randomly pursued based on the propensity functions.

We'll simulate the system for 10,000 seconds, and visualize what happens.

In [ ]:
history = simulate(State(10, 10), 10000.0, change, propensities)

This takes on average ~4.565 seconds to run. If you use `@btime` from Benchmark Tools, it'll take around 41 seconds to run. Note, this is a large unnormalized array (variable array length). I could go through and re-sample the array and match the times to a fixed sampling frequency, but for right now I'll just work with the entire array 😫

### Initial Results:

Plotting the production of A on a log scale, we can see some interesting things. Every now and then there appears to be a spike of about $10^6$ in the molecule number of A. I wonder what causes this? Additionally, this spike is not predicted by the deterministic dynamics of the system, what could it be?

Just empirically looking at this graph, I see that there appears to be about 5-6 spike events per 2,500s ~ 42 minutes


In [ ]:
gr()
cp = cgrad(:roma, 10, categorical = true, scale = :exp)

plot(
    [x[1] for x in history],
    [log.(x[2].A) for x in history],
    label = L"A",
    linewidth = 0.2,
    xlabel = L"t",
    ylabel = L"\log{\left(A(t)\right)}",
    legend = false,
    title = String(L"\textrm{Stochastic\,\, Resonance\,\, I} - A(t) \textrm{\,vs.\,} t"),
    palette = cp[1:4],
    background_color = :white
    
    
)

In [ ]:
plot(
    [log.(x[2].A) for x in history],
    [log.(x[2].B) for x in history],
    xlabel = L"\log{(A(t))}",
    ylabel = L"\log{(B(t))}",
    lw = 0.5,
    legend = false,
    title = String(L"\textrm{Stochastic\,\, Resonance\, I}: A \textrm{\,vs.\,} B"),
    palette = cp[8:end],
    background_color = :white

)

# To Do:
- Fourier stuff/coeffs 


# References
- [Approximation & Inference Methods in Biochemical Kinetics](https://iopscience.iop.org/article/10.1088/1751-8121/aa54d9/meta)
- [Practical Guide to Reaction Diffusion Systems](https://people.maths.ox.ac.uk/erban/Education/StochReacDiff.pdf)
- [Cool other paper I tried to recreate](https://www.pnas.org/doi/abs/10.1073/pnas.2303115120)
